In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
from keras import backend as K
#GPU utilize monitor
print(K.tensorflow_backend._get_available_gpus())

['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# load document
in_filename = 'train_set.txt'
doc = load_doc(in_filename)
#use first 1000000 lenth for training, make the model undercontrol,Unique Tokens will be 9395
doc = doc[0:1000000]
print(len(doc))
print(doc[:200])

1000000
"Here be our forecast"
"Traveling to have a business meet take the fun out of the trip Especially if you have to prepare a presentation I would suggest hold the business plan meet here then take a tri


In [5]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    tokens = doc.split()
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['be', 'our', 'to', 'have', 'a', 'business', 'meet', 'take', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'a', 'presentation', 'i', 'would', 'suggest', 'hold', 'the', 'business', 'plan', 'meet', 'here', 'then', 'take', 'a', 'trip', 'without', 'any', 'formal', 'business', 'meet', 'i', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'a', 'trip', 'be', 'even', 'desire', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meet', 'i', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'group', 'about', 'what', 'be', 'work', 'and', 'what', 'be', 'not', 'too', 'often', 'the', 'presenter', 'speak', 'and', 'the', 'others', 'be', 'quiet', 'just', 'wait', 'for', 'their', 'turn', 'the', 'meet', 'might', 'be', 'better', 'if', 'hold', 'in', 'a', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 179773


In [8]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
# save sequences to file
out_filename = 'email_sequences.txt'
save_doc(sequences, out_filename)

In [10]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [11]:
# load
in_filename = 'email_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [12]:
# integer encode sequences of words
tokenizer = Tokenizer(num_words = 12345)
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [13]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [14]:
X.shape

(179773, 50)

In [15]:
y.shape

(179773, 9396)

In [16]:
#partial validation data
x_val = X[:35000]
partial_x_train = X[35000:]

y_val = y[:35000]
partial_y_train = y[35000:]

In [17]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            469800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 9396)              948996    
Total params: 1,569,696
Trainable params: 1,569,696
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [19]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history = model.fit(partial_x_train, partial_y_train, 
                    batch_size=256, 
                    epochs=150, 
                    callbacks=callbacks, 
                    validation_data=(x_val, y_val),verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 144773 samples, validate on 35000 samples
Epoch 1/150
144773/144773 [==============================] - 99s 687us/step - loss: 7.0154 - acc: 0.0415 - val_loss: 6.5876 - val_acc: 0.0621
Epoch 2/150
144773/144773 [==============================] - 96s 661us/step - loss: 6.6147 - acc: 0.0621 - val_loss: 6.3915 - val_acc: 0.0849
Epoch 3/150
144773/144773 [==============================] - 94s 651us/step - loss: 6.3434 - acc: 0.0840 - val_loss: 5.8712 - val_acc: 0.1224
Epoch 4/150
144773/144773 [==============================] - 94s 649us/step - loss: 6.0888 - acc: 0.1052 - val_loss: 5.6417 - val_acc: 0.1500
Epoch 5/150
144773/144773 [==============================] - 94s 653us/step - loss: 5.8053 - acc: 0.1307 - val_loss: 5.4074 - val_acc: 0.1691
Epoch 6/150
144773/144773 [==============================] - 95s 654us/step - loss: 5.5518 - acc: 0.1475 - val_loss: 5.2493 - val_acc: 0.1805
Epoch 7/150
144773/144773 [======================

In [20]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

In [21]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

NameError: name 'history_dict' is not defined

<Figure size 432x288 with 0 Axes>

In [ ]:
# save the model to file
model.save('model_email_05920.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer05920.pkl', 'wb'))